In [1]:
from preamble import *
from preamble.dynesty.pooled_lw_rot import *

In [2]:
from functools import partial
from os.path import isfile

@partial(jax.jit, static_argnums=(1,))
def ln_like(θ, without=-1):
#     return sum(ln_like_chi2_2dof(m, o) for m, o in zip(model(θ), chunk_PS))
    return ln_like_chi2_2dof(chunk_model(ν, θ, without=without), PS)
reset = False

if (not isfile(checkpoint)) or reset:
    sampler = dynesty.DynamicNestedSampler(ln_like, prior_transform, Ndims, reflective=[0])
    sampler.run_nested()
    summary = summarise(sampler)
    np.save(checkpoint, {'summary': summary})
else:
    d = np.load(checkpoint, allow_pickle=True)[()]
    summary = d['summary']
    
from os import makedirs
from os.path import basename
figdir = f"figures/{basename(checkpoint).replace('.npy', '')}"
makedirs(figdir, exist_ok=True)

In [3]:
from scipy.optimize import minimize

# get ML solution

θ0 = summary['new_samples'][-1]
f = lambda *a: -ln_like(*a)
j0 = minimize(f, θ0, method='BFGS')
θ = j0['x']
q0 = j0['fun']
j0

No GPU/TPU found, falling back to CPU. (Set TF_CPP_MIN_LOG_LEVEL=0 and rerun for more info.)


  message: Optimization terminated successfully.
  success: True
   status: 0
      fun: 1488.612548828125
        x: [ 1.506e+00  5.996e-02 ...  7.682e-01  9.810e-02]
      nit: 0
      jac: [ 0.000e+00  0.000e+00 ...  0.000e+00  0.000e+00]
 hess_inv: [[1 0 ... 0 0]
            [0 1 ... 0 0]
            ...
            [0 0 ... 1 0]
            [0 0 ... 0 1]]
     nfev: 38
     njev: 1

In [4]:
from scipy.stats.distributions import chi2
for i, _ in enumerate(nu_guess):
    f = lambda *a: -ln_like(*a, without=i)/10000
    j = minimize(f, θ0, method='TNC')
    q = j['fun']*10000
    if np.isnan(q):
        q = -ln_like(θ, without=i)

    # q = 1/2 χ^2
    # Likelihood ratio is just exp(Δq)
    
    Λ = 2 * (q - q0)
    LR = np.exp(q0 - q)
    
    # χ2-distributed, with 2 degrees of freedom
    # (ν and A)
    p = chi2.sf(Λ, 2)

    print(f"{Λ:.3f} | {p:.3e}")

125.630 | 5.246e-28
50.836 | 9.143e-12
78.783 | 7.807e-18
10.043 | 6.593e-03
10.350 | 5.658e-03
23.046 | 9.899e-06
61.604 | 4.197e-14
51.437 | 6.770e-12
61.276 | 4.943e-14
18.667 | 8.841e-05
0.509 | 7.751e-01
20.520 | 3.501e-05
43.010 | 4.576e-10
73.093 | 1.343e-16
6.595 | 3.697e-02
